In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np


from sklearn.pipeline import Pipeline
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.base import clone
from sklearn.linear_model import Lasso, LogisticRegression, ElasticNet
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from scipy.stats import spearmanr
from sklearn.metrics import make_scorer

%config InlineBackend.figure_formats=['retina'] 

In [2]:
# Specimens and data files.
data_dir = "data/"
specimens_file = "subject_specimen.tsv"
pbmc_cell_freq_file = "pbmc_cell_frequency.tsv"
pbmc_cell_freq_normalized_file = "pbmc_cell_frequency_normalized.tsv"
pbmc_gene_expr_file = "pbmc_gene_expression_raw_count.tsv"
pbmc_gene_expr_raw_file = "pbmc_gene_expression_raw_count_raw_data.tsv"
plasma_ab_titer_file = "plasma_ab_titer.tsv"
plasma_ab_titer_normalized_file = "plasma_ab_titer_normalized.tsv"
plasma_cytokine_file = "plasma_cytokine_concentrations_by_olink.tsv"
t_cell_act_file = "t_cell_activation.tsv"
t_cell_pol_file = "t_cell_polarization.tsv"
th1_th2_pol_file = "Th1_Th2_polarization_ratio.tsv"
challenge_specimens_file = "challenge_subject_specimen.tsv"
tranining_specimens_file = "training_subject_specimen.tsv"

In [3]:
# Load batch corrected data into assay dictionary. Use batch corrected to avoid batch effects. 
assays = {}
assays["pbmc_cell_freq"] = pbmc_cell_freq_file
assays["pbmc_gene_expr"] = pbmc_gene_expr_file
assays["plasma_ab_titer"] = plasma_ab_titer_file
assays["plasma_cytokine"] = plasma_cytokine_file
assays["t_cell_act"] = t_cell_act_file
assays["t_cell_pol"] = t_cell_pol_file
assays["th1_th2_pol"] = th1_th2_pol_file

# Use pre batch corrected data for the FC calculations, since batch correction can change the fold change value.
assays["pbmc_cell_freq_normalized"] = pbmc_cell_freq_normalized_file
assays["pbmc_gene_expr_raw"] = pbmc_gene_expr_raw_file
assays["plasma_ab_titer_normalized"] = plasma_ab_titer_normalized_file

datasets = ["2020_dataset", "2021_dataset", "2022_dataset", "2023_dataset"]
training_datasets = ["2020_dataset", "2021_dataset", "2022_dataset"]

batch_corrected_assays = [
    "plasma_ab_titer",
    "plasma_cytokine",
    "pbmc_gene_expr",
    "pbmc_cell_freq",
    "t_cell_act",
    "t_cell_pol",
    "th1_th2_pol",
]

pre_batch_corrected_assays = ["pbmc_cell_freq_normalized", "pbmc_gene_expr_raw", "plasma_ab_titer_normalized"]

# All challenges.
challenges = [
    "1.1) IgG-PT-D14-titer-Rank",
    "1.2) IgG-PT-D14-FC-Rank",
    "2.1) Monocytes-D1-Rank",
    "2.2) Monocytes-D1-FC-Rank",
    "3.1) CCL3-D3-Rank",
    "3.2) CCL3-D3-FC-Rank",
    "4.1) IFNG/IL5-Polarization-D30-Rank",
]

# Features and assays for each challenge.
igg_feature_name = "IgG_PT"
igg_assay = "plasma_ab_titer"

monocytes_feature_name = "Monocytes"
monocytes_assay = "pbmc_cell_freq"

ccl3_feature_name = "ENSG00000277632.1"
ccl3_assay = "pbmc_gene_expr"

bonus_feature_name = "PT_P01579(IFNγ)/PT_P05113(IL5)"
bonus_assay = "th1_th2_pol"

# Random seed.
random_state = 42

In [4]:
# Get metadata about all the specimens.
specimens = pd.read_table(data_dir + specimens_file)
specimens = specimens.set_index("specimen_id")
challenge_specimens = pd.read_table(data_dir + challenge_specimens_file).set_index("specimen_id")
training_specimens = pd.read_table(data_dir + tranining_specimens_file).set_index("specimen_id")

# Get age of each specimen based on year of birth and date of boost.
def get_age(row):
    date_format = "%Y-%m-%d"
    birth = datetime.strptime(row["year_of_birth"], date_format)
    boost = datetime.strptime(row["date_of_boost"], date_format)
    return boost.year - birth.year

# Merge age with metadata about specimens.
challenge_specimens["age"] = challenge_specimens.apply(get_age, axis=1)
training_specimens["age"] = training_specimens.apply(get_age, axis=1)
all_specimens_age = pd.concat([challenge_specimens[["age"]], training_specimens[["age"]]])
specimens = specimens.merge(all_specimens_age, left_index=True, right_index=True, how="left")

In [19]:
# Load all assay data into a dictionary.
# Transpose the dataframes so that specimens are rows and features are columns.
assay_data = {}
feature_assay_dict = {}
for assay, file in assays.items():
    df = pd.read_table(data_dir + file)
    if assay != bonus_assay:
        # The bonus dataset does not require a transpose
        df = df.T
    else:
        df = df.set_index("specimen_id")
    df.index = df.index.astype("int64")
    df.index.name = "specimen_id"
    # Remove the dataset column if it exists, as it is not needed for analysis.
    if "dataset" in df.columns:
        df = df.drop("dataset", axis=1)
    # Create dictionary that maps features to assays.
    for column in df.columns:
        if column not in feature_assay_dict:
            feature_assay_dict[column] = assay
    assay_data[assay] = df

In [20]:
def print_assay_data_stats(timepoint):
    assay_data_stats = {}
    print(f"TIMEPOINT                 ", timepoint)
    for assay, data in assay_data.items():
        # Only consider batch corrected assays to see distribution of specimens across datasets and timepoints.
        if "_normalized" in assay or "_raw" in assay:
            continue
        merged_assay_data = data.merge(specimens, left_index=True, right_index=True, how="left")
        merged_assay_data = merged_assay_data[merged_assay_data["timepoint"] == timepoint].drop("timepoint", axis=1)
        counts = []
        for dataset in datasets:
            count = len(merged_assay_data[merged_assay_data["dataset"] == dataset])
            counts.append(count)
        assay_data_stats[assay] = counts

    for key, value in assay_data_stats.items():
        print(f"{key:<30}", value)

    subjects = []
    # Count number of subjects for each dataset at given timepoint (ignoring assay).
    for dataset in datasets:
        baseline_specimens = specimens[specimens["timepoint"] == timepoint]
        count = len(baseline_specimens[baseline_specimens["dataset"] == dataset])
        subjects.append(count)
    print(f"Subject count                 ", subjects)


print_assay_data_stats(0)
# print_assay_data_stats(-14)
# print_assay_data_stats(-15)
# print_assay_data_stats(-30)

TIMEPOINT                  0
pbmc_cell_freq                 [20, 33, 21, 48]
pbmc_gene_expr                 [36, 36, 21, 53]
plasma_ab_titer                [58, 33, 21, 54]
plasma_cytokine                [18, 36, 19, 32]
t_cell_act                     [0, 34, 20, 52]
t_cell_pol                     [0, 27, 16, 51]
th1_th2_pol                    [0, 27, 16, 51]
Subject count                  [60, 36, 21, 54]


In [21]:
def get_input_data(individual_assay_data, relevant_assays, relevant_datasets, timepoint):
    # Concatenate all relevant assays into a single input dataframe.
    input_data = pd.concat(
        (lambda d, k: [d[key] for key in k])(individual_assay_data, relevant_assays),
        axis=1,
    )
    input_data = input_data.merge(specimens, left_index=True, right_index=True, how="left")
    # Take only relevant datasets.
    input_data = input_data[input_data["dataset"].isin(relevant_datasets)]
    input_data = input_data.drop("date_of_boost", axis=1)

    # Convert infancy_vac and biological_sex to numerical features.
    label_encoder = LabelEncoder()
    input_data["infancy_vac"] = label_encoder.fit_transform(input_data["infancy_vac"])
    input_data["biological_sex"] = label_encoder.fit_transform(input_data["biological_sex"])

    # Take only the relevant timepoints.
    input_data = input_data[input_data["timepoint"] == timepoint].drop("timepoint", axis=1)
    input_data = input_data.set_index("subject_id")
    return input_data

In [22]:
def get_input_data_multiple_timepoints(individual_assay_data, relevant_assays, relevant_datasets, timepoints_dict):
    all_samples = pd.concat(
        (lambda d, k: [d[key] for key in k])(individual_assay_data, relevant_assays),
        axis=1,
    )

    # Merge with specimens to get metadata about each specimen.
    all_samples = all_samples.merge(specimens, left_index=True, right_index=True, how="left")
    all_samples = all_samples[all_samples["dataset"].isin(relevant_datasets)]
    all_samples = all_samples.drop("date_of_boost", axis=1)

    # Convert infancy_vac and biological_sex to numerical features.
    label_encoder = LabelEncoder()
    all_samples["infancy_vac"] = label_encoder.fit_transform(all_samples["infancy_vac"])
    all_samples["biological_sex"] = label_encoder.fit_transform(all_samples["biological_sex"])

    # Exclude meatadata columns not used by MOFA.
    exclude_columns = ["dataset", "infancy_vac", "biological_sex", "age"]

    first = True
    for timepoint, suffix in timepoints_dict.items():
        # Take data from the specified timepoint and drop the metadata columns.
        timepoint_data = all_samples[all_samples["timepoint"] == timepoint].drop("timepoint", axis=1)
        timepoint_data = timepoint_data.set_index("subject_id")
        updated_columns = []
        # Ensure columns are updated with the suffix for the timepoint, except for excluded columns.
        for col in timepoint_data.columns:
            if col in exclude_columns:
                updated_columns.append(col)
            else:
                updated_columns.append(f"{col}{suffix}")
        timepoint_data.columns = updated_columns
        if first:
            input_data = timepoint_data
        else:
            input_data = pd.concat(
                [
                    input_data,
                    timepoint_data.drop("age", axis=1)
                    .drop("dataset", axis=1)
                    .drop("infancy_vac", axis=1)
                    .drop("biological_sex", axis=1),
                ],
                axis=1,
            )
        first = False

    return input_data


In [23]:
def get_outcomes_dict(individual_assay_data, relevant_datasets):
    # Concat all batch corrected assays into a single input dataframe.
    input_data = pd.concat(
        (lambda d, k: [d[key] for key in k])(individual_assay_data, batch_corrected_assays),
        axis=1,
    )
    input_data = input_data.merge(specimens, left_index=True, right_index=True, how="left")
    input_data = input_data[input_data["dataset"].isin(relevant_datasets)].drop("dataset", axis=1)

    # Concat all pre batch corrected assays into a single dataframe.
    pre_batch_corrected_data = pd.concat(
        (lambda d, k: [d[key] for key in k])(individual_assay_data, pre_batch_corrected_assays),
        axis=1,
    )
    pre_batch_corrected_data = pre_batch_corrected_data.merge(specimens, left_index=True, right_index=True, how="left")
    pre_batch_corrected_data = pre_batch_corrected_data[
        pre_batch_corrected_data["dataset"].isin(relevant_datasets)
    ].drop("dataset", axis=1)

    # Create batch-corrected data for dataframes for each timepoint.
    timepoint_0 = input_data[input_data["timepoint"] == 0].set_index("subject_id")
    timepoint_1 = input_data[input_data["timepoint"] == 1].set_index("subject_id")
    timepoint_3 = input_data[input_data["timepoint"] == 3].set_index("subject_id")
    timepoint_14 = input_data[input_data["timepoint"] == 14].set_index("subject_id")
    timepoint_30 = input_data[input_data["timepoint"] == 30].set_index("subject_id")

    # Create pre-batch corrected data for each timepoint. 
    timepoint_0_pre_batch_corrected = pre_batch_corrected_data[pre_batch_corrected_data["timepoint"] == 0].set_index(
        "subject_id"
    )
    timepoint_1_pre_batch_corrected = pre_batch_corrected_data[pre_batch_corrected_data["timepoint"] == 1].set_index(
        "subject_id"
    )
    timepoint_3_pre_batch_corrected = pre_batch_corrected_data[pre_batch_corrected_data["timepoint"] == 3].set_index(
        "subject_id"
    )
    timepoint_14_pre_batch_corrected = pre_batch_corrected_data[pre_batch_corrected_data["timepoint"] == 14].set_index(
        "subject_id"
    )

    # Select features for each timepoint and add suffixes to distinguish between timepoints and batch corrected vs pre-batch corrected.
    timepoint_0_selected = timepoint_0[[igg_feature_name, monocytes_feature_name, ccl3_feature_name]].add_suffix("_0")
    timepoint_0_pre_batch_corrected_selected = timepoint_0_pre_batch_corrected[
        [igg_feature_name, monocytes_feature_name, ccl3_feature_name]
    ].add_suffix("_0P")
    timepoint_1_selected = timepoint_1[[monocytes_feature_name]]
    timepoint_1_pre_batch_corrected_selected = timepoint_1_pre_batch_corrected[[monocytes_feature_name]].add_suffix(
        "_P"
    )
    timepoint_3_selected = timepoint_3[[ccl3_feature_name]]
    timepoint_3_pre_batch_corrected_selected = timepoint_3_pre_batch_corrected[[ccl3_feature_name]].add_suffix("_P")
    timepoint_14_selected = timepoint_14[[igg_feature_name]]
    timepoint_14_pre_batch_corrected_selected = timepoint_14_pre_batch_corrected[[igg_feature_name]].add_suffix("_P")
    timepoint_30_selected = timepoint_30[[bonus_feature_name]]

    # Concat all selected feature data into a single dataframe.
    all_selected = pd.concat(
        [
            timepoint_0_selected,
            timepoint_0_pre_batch_corrected_selected,
            timepoint_1_selected,
            timepoint_1_pre_batch_corrected_selected,
            timepoint_3_selected,
            timepoint_3_pre_batch_corrected_selected,
            timepoint_14_selected,
            timepoint_14_pre_batch_corrected_selected,
            timepoint_30_selected,
        ],
        axis=1,
    )

    # Save outcome data using selected feature from each challenge. For FC challenges, calculate fold change using pre-batch corrected data.
    outcomes_dict = {}
    outcomes_dict[challenges[0]] = all_selected[igg_feature_name]
    outcomes_dict[challenges[1]] = all_selected[igg_feature_name + "_P"] / all_selected[igg_feature_name + "_0P"]
    outcomes_dict[challenges[1]].name = igg_feature_name + "_FC"

    outcomes_dict[challenges[2]] = all_selected[monocytes_feature_name]
    outcomes_dict[challenges[3]] = (
        all_selected[monocytes_feature_name + "_P"] / all_selected[monocytes_feature_name + "_0P"]
    )
    outcomes_dict[challenges[3]].name = monocytes_feature_name + "_FC"

    outcomes_dict[challenges[4]] = all_selected[ccl3_feature_name]
    outcomes_dict[challenges[5]] = all_selected[ccl3_feature_name + "_P"] / all_selected[ccl3_feature_name + "_0P"]
    outcomes_dict[challenges[5]].name = ccl3_feature_name + "_FC"

    outcomes_dict[challenges[6]] = all_selected[bonus_feature_name]

    return outcomes_dict

In [24]:
# Drops filters with percentage of missing values above a threshold.
def low_info_filter(df, threshold):
    missing_percentages = df.isnull().mean()
    return df.loc[:, missing_percentages <= threshold]

# Impute missing values using KNN imputer.
def impute_missing_values(df):
    # Initialize the KNN Imputer with k=3
    knn_imputer = KNNImputer(n_neighbors=3)

    # Fit and transform the data
    return pd.DataFrame(knn_imputer.fit_transform(df), columns=df.columns, index=df.index)

# Normalizes values in dataframe.
def scale_values(df, columns_to_skip):
    std_scaler = StandardScaler()
    df_scaled = df
    filtered_columns = []
    for col in df.columns:
        if col not in columns_to_skip:
            filtered_columns.append(col)
    
    df_scaled[filtered_columns] = std_scaler.fit_transform(df[filtered_columns])
    return df_scaled

# Normalizes values in series.
def scale_series(s):
    std_scaler = StandardScaler()
    data_normalized = std_scaler.fit_transform(s.values.reshape(-1, 1))
    return pd.Series(data_normalized.flatten(), index=s.index)

In [25]:
experiment = "experiment-mofa1"

# datasets_to_use = ["2022_dataset"]
# datasets_to_use = ["2021_dataset", "2022_dataset"]
datasets_to_use = ["2020_dataset", "2021_dataset", "2022_dataset"]

prune_assay = False
#prune_assay = True

input_data = get_input_data(assay_data, batch_corrected_assays, datasets_to_use, timepoint=0)
print("Input subjects: ", len(input_data))
print("Input features: ", len(input_data.columns))
assert len(input_data.columns.unique()) == len(input_data.columns)

outcomes_dict = get_outcomes_dict(assay_data, datasets_to_use)

# Remove specimens that do not have assay data if assay required for that challenge.
def prune_data_without_assay(input_data, feature, dataset):
    pruned_input_data = input_data[~(input_data[feature].isna() & (input_data["dataset"] == dataset))]
    return pruned_input_data

if prune_assay:
    input_data = prune_data_without_assay(input_data, monocytes_feature_name, "2020_dataset")

training_data = input_data.drop("dataset", axis=1)

print("Samples :", len(training_data))

Input subjects:  115
Input features:  6779
Samples : 115


In [26]:
# Drop features with many missing values, impute missing values and scale.
def preprocess_training_data(training_data, filter_threshold=1):
    training_data_filtered = low_info_filter(training_data, filter_threshold)
    print("Dropped", len(training_data.columns) - len(training_data_filtered.columns), "features")
    # for key in training_data.columns:
    #    if key not in training_data_filtered.columns:
    #        print("Dropped feature ", key)
    # training_data_scaled = scale_values(training_data_filtered, columns_to_skip=[])
    return training_data

training_data = training_data.drop('biological_sex', axis=1).drop('age', axis=1)
training_data = preprocess_training_data(training_data)


Dropped 0 features


In [27]:
# Resetting index to transform data into MOFA format.
training_data = training_data.reset_index()

In [28]:
# Rename subject_id to sample to match MOFA format.
training_data.rename(columns={'subject_id': 'sample'}, inplace=True)
training_data

,sample,IgG_PT,IgG_PRN,IgG_FHA,IgG1_PT,IgG1_PRN,IgG1_FHA,IgG1_FIM2/3,IgG1_TT,IgG1_DT,...,PT,TT,PHA_P01579,PHA_Q16552,PHA_P05113,PT_P01579,PT_Q16552,PT_P05113,PT_P01579(IFNγ)/PT_P05113(IL5),infancy_vac
0,1,2.979295,2.006372,26.636688,10.098853,2.044475,4.616537,0.155137,1.578561,3.176943,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,3,0.840128,5.248238,1.732962,-2.168427,3.008599,0.941351,-1.140584,1.495840,1.668517,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,4,1.609969,3.975809,1.696552,2.946572,2.451276,0.903670,2.298209,1.974648,2.538452,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,5,3.028724,3.717675,0.968178,10.351228,2.616303,4.300460,9.578256,1.753098,2.934753,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,6,0.124780,0.393764,1.190998,-4.949479,-0.055424,-0.407395,-0.604144,0.688148,-0.344389,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,115,6.051650,-1.904160,2.341280,9.861642,3.007505,1.557882,5.908761,2.076000,0.447946,...,2.000000,5.707317,1.097357,1.183491,0.482759,1.849495,0.297076,0.103448,17.878471,0
111,101,5.566932,0.518077,-1.601971,9.296850,1.278007,0.160909,-1.909336,0.906403,1.122905,...,2.129032,2.390244,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
112,82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.221429,0.494444,0.380207,1.281491,1.000000,1.000000,0.958377,0.127659,7.833350,0
113,87,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [29]:
# Melt the training data to transform it into MOFA format.
training_data = training_data.melt(id_vars=['sample', 'infancy_vac'])

In [30]:
# Rename variable column to feature to match MOFA format.
training_data.rename(columns={'variable': 'feature'}, inplace=True)
training_data

,sample,infancy_vac,feature,value
0,1,1,IgG_PT,2.979295
1,3,1,IgG_PT,0.840128
2,4,1,IgG_PT,1.609969
3,5,1,IgG_PT,3.028724
4,6,1,IgG_PT,0.124780
...,...,...,...,...
779120,115,0,PT_P01579(IFNγ)/PT_P05113(IL5),17.878471
779121,101,0,PT_P01579(IFNγ)/PT_P05113(IL5),NaN
779122,82,0,PT_P01579(IFNγ)/PT_P05113(IL5),7.833350
779123,87,0,PT_P01579(IFNγ)/PT_P05113(IL5),NaN


In [31]:
# Remove data type errors and ensure all strings are ASCII.
training_data['view'] = training_data['feature'].apply(lambda x: feature_assay_dict[x])
training_data['feature'] = training_data['feature'].astype(str).apply(lambda x: x.encode('ascii', errors='ignore').decode('ascii'))
training_data['sample'] = training_data['sample'].astype(str).apply(lambda x: x.encode('ascii', errors='ignore').decode('ascii'))
training_data['infancy_vac'] = training_data['infancy_vac'].astype(str)

In [32]:
# Create bins for outcome as MOFA requires categorical outcomes. Each bin is of equal range. 
bin_count = 8
bins = []
for i in range(0, bin_count):
    bins.append("b" + str(i))
outcomes_bins_dict = {}
training_data_dict = {}
for outcome in outcomes_dict:
    col_name = outcomes_dict[outcome].name
    outcomes_bins_dict[outcome] = pd.cut(outcomes_dict[outcome], bins=bin_count, labels=bins).reset_index().rename(columns = {'subject_id' : 'sample'})
    outcomes_bins_dict[outcome]['sample'] =  outcomes_bins_dict[outcome]['sample'].astype(str)
    training_data_dict[outcome] = pd.merge(training_data, outcomes_bins_dict[outcome], on='sample').rename(columns={col_name : 'group'}).dropna(subset = ['group'])
    training_data_dict[outcome]['group'] = training_data_dict[outcome]['group'].astype(str)

In [33]:
from mofapy2.run.entry_point import entry_point
import pandas as pd
import numpy as np

mofa_entry_point_dict = {}

# Run MOFA for each challenge and save results. 
for outcome in challenges:
    ent = entry_point()
    ent.set_data_options(
        scale_views = True
    )

    ent.set_data_df(training_data_dict[outcome])

    ent.set_model_options(
        factors = 2, 
        spikeslab_weights = True,
        ard_factors= True,
        ard_weights = True
    )
    ent.set_train_options(
        convergence_mode = "slow", 
        dropR2 = 0.001, 
        gpu_mode = True, 
        seed = 1
    )
    
    mofa_entry_point_dict[outcome] = ent
    ent.build()
    ent.run()
    ent.save(outfile="experiment-" + outcome[:3] + ".hdf5")



        #########################################################
        ###           __  __  ____  ______                    ### 
        ###          |  \/  |/ __ \|  ____/\    _             ### 
        ###          | \  / | |  | | |__ /  \ _| |_           ### 
        ###          | |\/| | |  | |  __/ /\ \_   _|          ###
        ###          | |  | | |__| | | / ____ \|_|            ###
        ###          |_|  |_|\____/|_|/_/    \_\              ###
        ###                                                   ### 
        ######################################################### 
       
 
        
Scaling views to unit variance...



Loaded group='b0' view='pbmc_cell_freq' with N=24 samples and D=39 features...
Loaded group='b0' view='pbmc_gene_expr' with N=24 samples and D=6650 features...
Loaded group='b0' view='plasma_ab_titer' with N=24 samples and D=31 features...
Loaded group='b0' view='plasma_cytokine' with N=24 samples and D=45 features...
Loaded group='b0' view='

In [34]:
import mofax as mfx
mofax_dict = {}
for outcome in challenges:
    mofax_dict[outcome] = mfx.mofa_model("experiment-" + outcome[:3] + ".hdf5")

In [35]:
# Print number of factors and important features for each challenge.
for outcome, model in mofax_dict.items():
    print("Challenge = ", outcome)
    model = mofax_dict[outcome]
    num_factors = len(model.factors)
    print("FACTORS = ", num_factors)
    imp_features = []
    for view in model.views:
        print("VIEW = ", view)
        w = model.get_weights(df=True, views = view)
        for factor in w.columns:
            factor_weights = w[factor]
            sorted_factor_weights = factor_weights.abs().sort_values(ascending=False)
            sorted_with_signs = factor_weights.loc[sorted_factor_weights.index]
            for feature in sorted_with_signs[:4].index:
                imp_features.append(feature)
    


Challenge =  1.1) IgG-PT-D14-titer-Rank
FACTORS =  8
VIEW =  pbmc_cell_freq
VIEW =  pbmc_gene_expr
VIEW =  plasma_ab_titer
VIEW =  plasma_cytokine
VIEW =  t_cell_act
VIEW =  t_cell_pol
VIEW =  th1_th2_pol
Challenge =  1.2) IgG-PT-D14-FC-Rank
FACTORS =  7
VIEW =  pbmc_cell_freq
VIEW =  pbmc_gene_expr
VIEW =  plasma_ab_titer
VIEW =  plasma_cytokine
VIEW =  t_cell_act
VIEW =  t_cell_pol
VIEW =  th1_th2_pol
Challenge =  2.1) Monocytes-D1-Rank
FACTORS =  8
VIEW =  pbmc_cell_freq
VIEW =  pbmc_gene_expr
VIEW =  plasma_ab_titer
VIEW =  plasma_cytokine
VIEW =  t_cell_act
VIEW =  t_cell_pol
VIEW =  th1_th2_pol
Challenge =  2.2) Monocytes-D1-FC-Rank
FACTORS =  8
VIEW =  pbmc_cell_freq
VIEW =  pbmc_gene_expr
VIEW =  plasma_ab_titer
VIEW =  plasma_cytokine
VIEW =  t_cell_act
VIEW =  t_cell_pol
VIEW =  th1_th2_pol
Challenge =  3.1) CCL3-D3-Rank
FACTORS =  7
VIEW =  pbmc_cell_freq
VIEW =  pbmc_gene_expr
VIEW =  plasma_ab_titer
VIEW =  plasma_cytokine
VIEW =  t_cell_act
VIEW =  t_cell_pol
VIEW =  th1_